In [4]:
import transformers
import pandas as pd
import json
import os
from tqdm import tqdm 

In [5]:
outerFolder = "/shared/3/projects/benlitterer/agendaSetting/testingHtml"

dfList = []
for folderNum in tqdm(os.listdir(outerFolder), total=len(os.listdir(outerFolder))): 
    innerFolder = outerFolder + "/" + folderNum 
    for innerFileName in os.listdir(innerFolder): 
        innerPathName = outerFolder + "/" + folderNum + "/" + innerFileName
        if "json" in innerPathName: 
            with open(innerPathName, "r") as f: 
                data = json.loads(f.read())
            df = pd.json_normalize(data)
            df["id"] = innerFileName.split(".")[0]
            dfList.append(df)
        

100%|██████████| 100/100 [01:02<00:00,  1.61it/s]


In [7]:
fullDf = pd.concat(dfList).reset_index(drop=True)

In [8]:
leanDf = fullDf[["id", "title", "url", "source_url", "text"]]

In [75]:
groundTruths = pd.read_csv("/shared/3/projects/benlitterer/agendaSetting/final_evaluation_data.csv")
print(groundTruths.shape)
groundTruths.head(3)

(4902, 14)


,url1_lang,url2_lang,pair_id,link1,link2,ia_link1,ia_link2,GEO,ENT,TIME,NAR,Overall,STYLE,TONE
0,en,en,1484189203_1484121193,https://wsvn.com/news/local/broward/police-2-m...,https://wsvn.com/news/local/no-swim-advisory-l...,https://web.archive.org/web/https://wsvn.com/n...,https://web.archive.org/web/https://wsvn.com/n...,1.5,4.0,2.0,4.0,3.5,1.0,1.500000
1,en,en,1484011097_1484011106,https://www.zdnet.com/article/autoclerk-databa...,https://securityboulevard.com/2019/10/best-wes...,https://web.archive.org/web/https://www.zdnet....,https://web.archive.org/web/https://securitybo...,1.0,2.0,1.0,1.0,1.0,3.5,2.500000
2,en,en,1484039488_1484261803,https://www.presstelegram.com/2019/12/31/ducks...,https://boingboing.net/2020/01/01/woody-guthri...,https://web.archive.org/web/https://www.presst...,https://web.archive.org/web/https://boingboing...,5.0,4.0,3.0,4.0,4.0,4.0,3.666667


In [76]:
#get just the english pairs 
groundTruths.loc[(groundTruths["url1_lang"] == "en") & (groundTruths["url2_lang"] == "en")].shape

(236, 14)

In [77]:
def splitIds(inStr): 
    return inStr.split("_")
#groundTruths[["id1", "id2"]] = groundTruths["pair_id"].apply(splitIds)
groundTruths["id1"] = [item[0] for item in groundTruths["pair_id"].apply(splitIds)]
groundTruths["id2"] = [item[1] for item in groundTruths["pair_id"].apply(splitIds)]

In [78]:
#This used to work but stopped working potentially due to pandas version error? 
#groundTruths[["id1", "id2"]] = groundTruths["pair_id"].str.split("_", expand=True)

In [79]:
leanDf = leanDf[["id", "text", "title"]]

In [80]:
#merge text for the first id, id1
leanDf1 = leanDf.rename(columns = {"id":"id1","text":"text1", "title":"title1"})
firstMerge = pd.merge(groundTruths, leanDf1, how="left", on="id1")
firstMerge.head(3)

,url1_lang,url2_lang,pair_id,link1,link2,ia_link1,ia_link2,GEO,ENT,TIME,NAR,Overall,STYLE,TONE,id1,id2,text1,title1
0,en,en,1484189203_1484121193,https://wsvn.com/news/local/broward/police-2-m...,https://wsvn.com/news/local/no-swim-advisory-l...,https://web.archive.org/web/https://wsvn.com/n...,https://web.archive.org/web/https://wsvn.com/n...,1.5,4.0,2.0,4.0,3.5,1.0,1.500000,1484189203,1484121193,"DAVIE, FLA. (WSVN) - Police need help catching...",Police: 2 men stole tools from Lowe’s in Davie
1,en,en,1484011097_1484011106,https://www.zdnet.com/article/autoclerk-databa...,https://securityboulevard.com/2019/10/best-wes...,https://web.archive.org/web/https://www.zdnet....,https://web.archive.org/web/https://securitybo...,1.0,2.0,1.0,1.0,1.0,3.5,2.500000,1484011097,1484011106,Govt officials confirm Trump can block US comp...,"Open database leaked 179GB in customer, US gov..."
2,en,en,1484039488_1484261803,https://www.presstelegram.com/2019/12/31/ducks...,https://boingboing.net/2020/01/01/woody-guthri...,https://web.archive.org/web/https://www.presst...,https://web.archive.org/web/https://boingboing...,5.0,4.0,3.0,4.0,4.0,4.0,3.666667,1484039488,1484261803,"Ducks defenseman Erik Gudbranson, left, knocks...",Ducks are own worst enemies in sloppy loss in ...


In [81]:
firstMerge[(firstMerge["url1_lang"] == "en") & (firstMerge["url2_lang"] == "en")].shape

(236, 18)

In [82]:
leanDf2 = leanDf.rename(columns = {"id":"id2","text":"text2","title":"title2" })
merged = pd.merge(firstMerge, leanDf2, how="left", on="id2")
merged.head(3)

,url1_lang,url2_lang,pair_id,link1,link2,ia_link1,ia_link2,GEO,ENT,TIME,NAR,Overall,STYLE,TONE,id1,id2,text1,title1,text2,title2
0,en,en,1484189203_1484121193,https://wsvn.com/news/local/broward/police-2-m...,https://wsvn.com/news/local/no-swim-advisory-l...,https://web.archive.org/web/https://wsvn.com/n...,https://web.archive.org/web/https://wsvn.com/n...,1.5,4.0,2.0,4.0,3.5,1.0,1.500000,1484189203,1484121193,"DAVIE, FLA. (WSVN) - Police need help catching...",Police: 2 men stole tools from Lowe’s in Davie,"DEERFIELD BEACH, FLA. (WSVN) - A no-swim advis...",No-swim advisory lifted for Deerfield Beach Pier
1,en,en,1484011097_1484011106,https://www.zdnet.com/article/autoclerk-databa...,https://securityboulevard.com/2019/10/best-wes...,https://web.archive.org/web/https://www.zdnet....,https://web.archive.org/web/https://securitybo...,1.0,2.0,1.0,1.0,1.0,3.5,2.500000,1484011097,1484011106,Govt officials confirm Trump can block US comp...,"Open database leaked 179GB in customer, US gov...",The latest huge unsecured cloud storage find i...,Best Western’s Massive Data Leak: 179GB Amazon...
2,en,en,1484039488_1484261803,https://www.presstelegram.com/2019/12/31/ducks...,https://boingboing.net/2020/01/01/woody-guthri...,https://web.archive.org/web/https://www.presst...,https://web.archive.org/web/https://boingboing...,5.0,4.0,3.0,4.0,4.0,4.0,3.666667,1484039488,1484261803,"Ducks defenseman Erik Gudbranson, left, knocks...",Ducks are own worst enemies in sloppy loss in ...,Woody Guthrie's 1943 New Year's Resolutions ar...,Woody Guthrie's 1943 New Year's Resolutions ar...


In [83]:
merged[(merged["url1_lang"] == "en") & (merged["url2_lang"] == "en")]

,url1_lang,url2_lang,pair_id,link1,link2,ia_link1,ia_link2,GEO,ENT,TIME,NAR,Overall,STYLE,TONE,id1,id2,text1,title1,text2,title2
0,en,en,1484189203_1484121193,https://wsvn.com/news/local/broward/police-2-m...,https://wsvn.com/news/local/no-swim-advisory-l...,https://web.archive.org/web/https://wsvn.com/n...,https://web.archive.org/web/https://wsvn.com/n...,1.5,4.0,2.0,4.0,3.5,1.0,1.500000,1484189203,1484121193,"DAVIE, FLA. (WSVN) - Police need help catching...",Police: 2 men stole tools from Lowe’s in Davie,"DEERFIELD BEACH, FLA. (WSVN) - A no-swim advis...",No-swim advisory lifted for Deerfield Beach Pier
1,en,en,1484011097_1484011106,https://www.zdnet.com/article/autoclerk-databa...,https://securityboulevard.com/2019/10/best-wes...,https://web.archive.org/web/https://www.zdnet....,https://web.archive.org/web/https://securitybo...,1.0,2.0,1.0,1.0,1.0,3.5,2.500000,1484011097,1484011106,Govt officials confirm Trump can block US comp...,"Open database leaked 179GB in customer, US gov...",The latest huge unsecured cloud storage find i...,Best Western’s Massive Data Leak: 179GB Amazon...
2,en,en,1484039488_1484261803,https://www.presstelegram.com/2019/12/31/ducks...,https://boingboing.net/2020/01/01/woody-guthri...,https://web.archive.org/web/https://www.presst...,https://web.archive.org/web/https://boingboing...,5.0,4.0,3.0,4.0,4.0,4.0,3.666667,1484039488,1484261803,"Ducks defenseman Erik Gudbranson, left, knocks...",Ducks are own worst enemies in sloppy loss in ...,Woody Guthrie's 1943 New Year's Resolutions ar...,Woody Guthrie's 1943 New Year's Resolutions ar...
3,en,en,1484332324_1484796748,https://www.financialexpress.com/india-news/we...,https://www.news18.com/news/india/not-the-firs...,https://web.archive.org/web/https://www.financ...,https://web.archive.org/web/https://www.news18...,1.0,1.5,1.0,1.5,1.5,1.5,2.000000,1484332324,1484796748,The West Bengal government’s proposal was reje...,Another Bengal vs Centre tussle? Govt rejects ...,Mumbai: The NCP and Shiv Sena on Thursday targ...,'Congress Rejected 7 Times': BJP's Reminder as...
4,en,en,1484012256_1484419682,https://www.birminghammail.co.uk/whats-on/musi...,http://m.fightbacknews.org/2020/1/1/top-20-fil...,https://web.archive.org/web/https://www.birmin...,https://web.archive.org/web/http://m.fightback...,4.0,4.0,1.0,2.5,4.0,2.5,2.500000,1484012256,1484419682,The video will start in 8 Cancel\n\nSign up to...,Bars and clubs you loved and lost this decade ...,"Jacksonville, FL - I'm not sure how we'll look...",Top 20 films of the 2010s
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4555,en,en,1492929930_1492536485,https://www.thehindu.com/news/cities/bangalore...,https://www.thehindu.com/news/national/kerala/...,https://web.archive.org/web/https://www.thehin...,https://web.archive.org/web/https://www.thehin...,2.0,4.0,1.0,4.0,4.0,1.0,2.000000,1492929930,1492536485,"Traders and officials, going by the ground rep...","From highs of ₹200, onion prices crash to the ...",Death at Government MCH at Manjeri\n\nDeath at...,Man dies due to alleged medical negligence
4556,en,en,1576323732_1576233684,https://www.irishpost.com/news/tim-brooke-tayl...,https://thenewdaily.com.au/entertainment/2020/...,https://web.archive.org/web/https://www.irishp...,https://web.archive.org/web/https://thenewdail...,1.0,2.0,1.0,1.0,1.0,1.0,1.000000,1576323732,1576233684,"TIM BROOKE-TAYLOR, who was best known as part ...","Tim Brooke-Taylor, star of 'The Goodies', dies...","British comedian and actor Tim Brooke-Taylor, ...",The Goodies’ Tim Brooke-Taylor dies of coronav...
4557,en,en,1587362537_1585362092,https://www.federalwaymirror.com/news/i-miss-y...,https://www.federalwaymirror.com/news/police-i...,https://web.archive.org/web/https://www.federa...,https://web.archive.org/web/https://www.federa...,1.0,1.0,1.0,2.0,2.0,1.0,1.000000,1587362537,1585362092,A handwritten note at the memorial for Damien ...,Shooting victim identified: ‘I miss you my boy’,Police responde

In [72]:
#mergedFull = merged.dropna(subset=["text1", "text2"])

In [66]:
mergedFull = mergedFull.dropna(subset=["title1", "title2"])

In [74]:
mergedFull[(mergedFull["url1_lang"] == "en") & (mergedFull["url2_lang"] == "en")].shape

(235, 20)

In [70]:
mergedEn = mergedFull[(mergedFull["url1_lang"] == "en") & (mergedFull["url2_lang"] == "en")]
mergedEn.shape

(235, 20)

In [71]:
mergedEn.to_csv("/shared/3/projects/benlitterer/localNews/NetworkMVP/enTestData.csv", sep="\t")